# 네이버 자동차 리뷰 분석 및 개선

In [1]:
# from bs4 import BeautifulSoup ## 안돼서 사용안함
import requests
import pandas as pd
import time
import selenium 

print(f'selenium v.{selenium.__version__}')

selenium v.4.38.0


### 정적으로는 리뷰를 가져올 수 없음 -> selenium을 사용해야함

In [2]:
#### ========================================================
#### 이렇게 진행해보니, 숨겨놔서 select로 불러올 수 없음
#### -> Selenium 사용해야함
#### ========================================================
# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
# }
# ## 사이트 주소로 객체 만드는 함수
# url = "https://search.naver.com/search.naver?where=nexearch&qvt=0&pkid=128&mra=bjg1&sm=tab_etc&os=40311477&query=2026%20%ED%98%BC%EB%8B%A4%20CR-V%20%ED%95%98%EC%9D%B4%EB%B8%8C%EB%A6%AC%EB%93%9C%20%ED%8F%89%EA%B0%80"  # 평가 페이지 링크

# res = requests.get(url, headers=HEADERS)
# res.raise_for_status()
# soup = BeautifulSoup(res.text, "html.parser")
# soup.select('div.u_cbox_content_wrap ul > li.u_cbox_comment')
# soup

### selenium 시작

In [3]:
## - 모듈 로딩 
from selenium import webdriver # 웹 브라우저(Chrome)를 자동으로 켜서 조작하기 위한 라이브러리
from selenium.webdriver.common.by import By # HTML 요소를 찾을 때 사용 (id, class, css 선택자 등)
from selenium.webdriver.chrome.service import Service # 크롬 드라이버(브라우저 조종 프로그램)를 연결해 주는 도구
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 자동으로 다운로드해 주는 라이브러리

# 웹 페이지가 뜰 때까지 / 특정 요소가 나올 때까지 기다려주는 도구
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# driver = webdriver.Chrome()

# ## 예시 차량 평가 페이지 -> 바로 열기
# driver.get('https://search.naver.com/search.naver?where=nexearch&qvt=0&pkid=128&mra=bjg1&sm=tab_etc&os=40299089&query=2026%20%EB%A9%94%EB%A5%B4%EC%84%B8%EB%8D%B0%EC%8A%A4-%EB%B2%A4%EC%B8%A0%20GLE%20%ED%8F%89%EA%B0%80')

# print(driver.title) 

In [5]:
# =========================================
# 크롬 드라이버 생성 함수
# =========================================
def create_driver():
    """Selenium용 크롬 드라이버를 생성해서 반환한다."""
    options = webdriver.ChromeOptions()
    # 창 안 띄우고 돌리고 싶으면 아래 주석 해제
    # options.add_argument("--headless=new")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

In [6]:
# =========================================
## 리뷰 총 개수 확인하는 함수
# =========================================
def get_total_review_count(driver):
    try:
        elem = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "span.rating_people > strong")
            )
        )
        total = int(elem.text.strip())
        return total
    except:
        return 0


In [7]:
def load_all_reviews_until_full(driver, car_name=""):
    """
    총 리뷰 개수(total_count)를 가져온 후,
    실제 로딩된 리뷰 블록 개수(loaded_count)가 total_count에 도달할 때까지 스크롤.
    """

    # 스크롤 박스 찾기
    scroll_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div._scroll_container")
        )
    )

    # 총 리뷰 개수 가져오기
    total_count = get_total_review_count(driver)
    print(f"[{car_name}] 총 리뷰 수: {total_count}")

    # 3) 로딩된 댓글 개수 확인하며 스크롤 반복
    last_loaded = 0
    same_round = 0

    for _ in range(50):  # 최대 50번 스크롤 시도
        # 로딩된 리뷰 수
        review_blocks = scroll_box.find_elements(
            By.CSS_SELECTOR, "div.u_cbox_comment_box"
        )
        loaded_count = len(review_blocks)

        print(f"현재 로딩된 리뷰 수: {loaded_count} / {total_count}")

        # 이미 다 로딩함 → 중지
        if loaded_count >= total_count:
            print("모든 리뷰 로딩 완료!")
            break

        # 변화가 없음 → 잠깐 기다리고 다시 시도
        if loaded_count == last_loaded:
            same_round += 1
        else:
            same_round = 0

        if same_round >= 3:
            print("3번 연속 리뷰 증가 없음 → 중지(로딩 끝난 것으로 판단)")
            break

        # 스크롤 실행
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight;",
            scroll_box
        )

        last_loaded = loaded_count
        time.sleep(1.1)

    # 로딩 완료 후 리뷰 element 리스트 반환
    return scroll_box.find_elements(By.CSS_SELECTOR, "div.u_cbox_comment_box")


In [ ]:
# =========================================
## 리뷰에서 차이름, 평가점수, 리뷰 내용 도출해서 append하는 함수
# =========================================
def scroll_and_collect_reviews(driver, car_name=""):
    review_blocks = load_all_reviews_until_full(driver, car_name)
    print(driver, car_name)

    data = []
    for rb in review_blocks:
        try:
            score = rb.find_element(By.CSS_SELECTOR, "span.u_cbox_multirating_totalcount_value").text.strip()
        except:
            score = None

        try:
            content = rb.find_element(By.CSS_SELECTOR, "div.u_cbox_text_wrap").text.strip()
        except:
            content = ""

        data.append({
            "car": car_name,
            "score": score,
            "content": content
        })

    print(f"[{car_name}] 최종 저장 리뷰 수: {len(data)}")
    return data


In [9]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def load_all_reviews_with_jiggle(driver, car_name=""):
    """
    평가 탭의 댓글 영역을 끝까지 스크롤하면서
    리뷰 블록(div.u_cbox_comment_box)을 최대한 많이 로딩해오는 함수
    """
    # 1) 스크롤 박스 찾기
    scroll_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div._scroll_container")
        )
    )

    last_count = 0
    stagnant_rounds = 0
    MAX_ROUNDS = 40

    for _ in range(MAX_ROUNDS):
        review_blocks = scroll_box.find_elements(
            By.CSS_SELECTOR, "div.u_cbox_comment_box"
        )
        cur_count = len(review_blocks)
        print(f"[{car_name}] 현재 리뷰 수: {cur_count}")

        if cur_count == last_count:
            # 살짝 위로
            driver.execute_script(
                "arguments[0].scrollTop = arguments[0].scrollTop - 120;",
                scroll_box
            )
            time.sleep(0.3)
            # 다시 아래로
            driver.execute_script(
                "arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].clientHeight;",
                scroll_box
            )
            time.sleep(0.8)

            review_blocks_after = scroll_box.find_elements(
                By.CSS_SELECTOR, "div.u_cbox_comment_box"
            )
            new_count = len(review_blocks_after)
            print(f"[{car_name}] joggle 후 리뷰 수: {new_count}")

            if new_count == cur_count:
                stagnant_rounds += 1
            else:
                stagnant_rounds = 0
                cur_count = new_count
                review_blocks = review_blocks_after
        else:
            stagnant_rounds = 0

        if stagnant_rounds >= 3:
            print(f"[{car_name}] 3번 연속 증가 없음 → 스크롤 종료")
            break

        last_count = cur_count

        # 기본 아래 스크롤
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].clientHeight;",
            scroll_box
        )
        time.sleep(0.7)

    final_blocks = scroll_box.find_elements(
        By.CSS_SELECTOR, "div.u_cbox_comment_box"
    )
    print(f"[{car_name}] 최종 리뷰 수: {len(final_blocks)}")
    return final_blocks


In [ ]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException


def go_to_eval_and_collect(driver, car_name=""):
    data = []

    # 1) 상단 탭 영역 컨테이너 찾기
    try:
        tab_container = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR,
                 "#root > section > div.top_wrap > div.nav_tab_area > div > div > div > div")
            )
        )
    except TimeoutException:
        print(f"[{car_name}] 상단 탭 영역 없음 → 평가 스킵")
        return data

    # 2) 탭 영역 안에서 span들을 보고 '평가' 텍스트 찾기
    eval_tab = None

    spans = tab_container.find_elements(By.CSS_SELECTOR, "div > a > span")
    print("탭 텍스트들:", [s.text.strip() for s in spans])  # 디버깅용

    for s in spans:
        txt = s.text.strip()
        if "평가" in txt:
            # span의 부모 <a> 요소를 찾아서 eval_tab에 저장
            eval_tab = s.find_element(By.XPATH, "..")
            break  # 찾았으면 더 돌 필요 없음

    # for문을 다 돌았는데도 평가 탭을 못 찾은 경우
    if eval_tab is None:
        print(f"[{car_name}] 탭 영역 안에 '평가' 없음 → 스킵")
        return data

    # 3) 평가 탭 클릭
    try:
        driver.execute_script("arguments[0].click();", eval_tab)
        time.sleep(1.5)  # 평가 탭 내용 로딩 대기
    except Exception as e:
        print(f"[{car_name}] '평가' 탭 클릭 실패 → 스킵 ({e})")
        return data

    # 4) 여기서부터는 댓글 스크롤 + 리뷰 수집
    #    (load_all_reviews_with_jiggle 를 이미 만들었다고 가정)

    try:
        review_blocks = load_all_reviews_until_full(driver, car_name)
    except Exception as e:
        print(f"[{car_name}] 리뷰 스크롤 중 에러: {e}")
        return data

    for rb in review_blocks:
        try:
            score = rb.find_element(
                By.CSS_SELECTOR, "span.rating_score"
            ).text.strip()
        except:
            score = None

        try:
            content = rb.find_element(
                By.CSS_SELECTOR, "div.u_cbox_text_wrap"
            ).text.strip()
        except:
            content = ""

        data.append({
            "car": car_name,
            "score": score,
            "content": content
        })

    print(f"[{car_name}] 최종 저장 리뷰 수: {len(data)}")
    return data


# -------------------------------------

In [11]:
# =========================================
## 차량 이름 출력 함수
# =========================================
CAR_SELECTOR = "div.list_info._panel div.info_area > strong > a"

def collect_car_links_on_page(driver):
    """현재 페이지에서 (차이름, href) 리스트를 뽑음"""
    elems = driver.find_elements(By.CSS_SELECTOR, CAR_SELECTOR)
    results = []
    for e in elems:
        name = e.text.strip()
        href = e.get_attribute("href")
        if href:
            results.append((name, href))
    return results


In [12]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# =========================================
## 오른쪽 화살표 클릭 함수
# =========================================

NEXT_BTN_SELECTOR = "div.cm_paging_area.no_margin a.pg_next"

def go_next_page(driver):
    """오른쪽 화살표(다음 페이지) 버튼을 눌러 다음 페이지로 이동. 성공하면 True, 더 이상 없으면 False."""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, NEXT_BTN_SELECTOR)

        # class 에 'disabled' 같은 게 붙는지 체크 (붙으면 더 이상 못 감)
        cls = next_btn.get_attribute("class") or ""
        if "disabled" in cls:
            return False

        # 현재 첫 번째 자동차 이름을 저장해둠 (페이지가 바뀌었는지 확인용)
        try:
            first_name = driver.find_element(By.CSS_SELECTOR, CAR_SELECTOR).text.strip()
        except:
            first_name = None

        # JS로 클릭 실행 (내부 자바스크립트 함수 호출됨)
        driver.execute_script("arguments[0].click();", next_btn)

        # 페이지 내용이 바뀔 때까지 잠깐 대기
        if first_name:
            WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element(
                    (By.CSS_SELECTOR, CAR_SELECTOR),
                    ""
                )
            )
        else:
            # 첫 요소가 없었다면 그냥 잠깐 sleep
            time.sleep(1.5)

        return True

    except NoSuchElementException:
        # next 버튼 자체가 없으면 끝
        return False
    except TimeoutException:
        # 내용이 안 바뀌면 더 진행하지 않음
        return False


In [13]:
# =========================================
## 페이지 넘기면서 페이지 들어가고 평가 들어가는 함수
# =========================================

def crawl_all_cars(start_url, max_pages=1):
    """
    start_url: '자동차' 검색 결과 페이지 URL
    max_pages: 최대 몇 페이지까지 넘길지 (테스트용)
    """

    driver = create_driver()
    driver.get(start_url)
    # time.sleep(3)

    all_links = []   # (차이름, href) 모으는 곳
    seen = set()     # 중복 제거용

    current_page = 1
    while current_page <= max_pages:
        print(f"\n=== {current_page} 페이지 수집 중 ===")

        # 현재 페이지에서 자동차 링크 수집
        page_links = collect_car_links_on_page(driver)
        print("이 페이지에서 찾은 차 개수:", len(page_links))

        for name, href in page_links:
            if href not in seen:
                seen.add(href)
                all_links.append((name, href))

        # 다음 페이지로 이동 시도
        moved = go_next_page(driver)
        if not moved:
            print("더 이상 다음 페이지가 없음 → 종료")
            break

        current_page += 1
        # time.sleep(2)

    print("\n총 수집된 차량 링크 수:", len(all_links))

    # 여기서 all_links 를 가지고 각 차량 상세 페이지 → 평가 탭 → 리뷰 수집
    all_reviews = []

    for idx, (car_name, href) in enumerate(all_links, start=1):
        print(f"\n===== [{idx}/{len(all_links)}] {car_name} ({href}) =====")
        try:
            driver.get(href)
            # time.sleep(2)
            car_reviews = go_to_eval_and_collect(driver)
            all_reviews.extend(car_reviews)
        except Exception as e:
            print("[ERROR] 차량 상세 페이지 처리 중 에러:", e)
            continue

    driver.quit()
    return all_reviews


In [ ]:
# =========================================
# 메인 실행부
# =========================================
if __name__ == "__main__":
    START_URL = (
        'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%9E%90%EB%8F%99%EC%B0%A8&ackey=j1rh28hp'
    )

    # '자동차' 검색 페이지에서 최대 10개 모델만 테스트
    reviews = crawl_all_cars(START_URL)

    # pandas DataFrame으로 변환 후 CSV 저장
    df = pd.DataFrame(reviews)
    df.to_csv("naver_car_reviews_all_final.csv", index=False, encoding="utf-8-sig")

    print("\n전체 저장 완료! 총 리뷰 수:", len(df))


=== 1 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 8

총 수집된 차량 링크 수: 8

===== [1/8] 2026 혼다 CR-V 하이브리드 (https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjg1&pkid=128&os=40311477&qvt=0&query=2026%20%ED%98%BC%EB%8B%A4%20CR-V%20%ED%95%98%EC%9D%B4%EB%B8%8C%EB%A6%AC%EB%93%9C) =====
탭 텍스트들: ['요약', '견적', '정보', '제원', '중고시세', '포토', '평가', '정비', '비교']
[] 현재 리뷰 수: 10
[] 현재 리뷰 수: 10
[] joggle 후 리뷰 수: 10
[] 현재 리뷰 수: 20
[] 현재 리뷰 수: 20
[] joggle 후 리뷰 수: 20
[] 현재 리뷰 수: 20
[] joggle 후 리뷰 수: 30
[] 현재 리뷰 수: 30
[] joggle 후 리뷰 수: 30
[] 현재 리뷰 수: 30
[] joggle 후 리뷰 수: 40
[] 현재 리뷰 수: 40
[] joggle 후 리뷰 수: 40
[] 현재 리뷰 수: 40
[] joggle 후 리뷰 수: 40
[] 현재 리뷰 수: 50
[] 현재 리뷰 수: 50
[] joggle 후 리뷰 수: 50
[] 현재 리뷰 수: 50
[] joggle 후 리뷰 수: 60
[] 현재 리뷰 수: 60
[] joggle 후 리뷰 수: 60
[] 현재 리뷰 수: 60
[] joggle 후 리뷰 수: 61
[] 현재 리뷰 수: 61
[] joggle 후 리뷰 수: 61
[] 현재 리뷰 수: 61
[] joggle 후 리뷰 수: 61
[] 현재 리뷰 수: 61
[] joggle 후 리뷰 수: 61
[] 3번 연속 증가 없음 → 스크롤 종료
[] 최종 리뷰 수: 61
[] 최종 저장 리뷰 수: 61

===== [2/8] 2026 토요타 프리우스 플러그... (https://se

# -----------------------------------------------
